## Run the agent with loaded Q-Table

In [1]:
import gym
import numpy as np
from gym.envs.registration import register
import matplotlib.pyplot as plt

from environment.ResourceManagerEnvironment_v1_8 import ResourceManagerEnv
from agent.QLearning import QLearningAgent

In [2]:
register(
    id='Resource-Manager-v1-8',
    entry_point='environment.ResourceManagerEnvironment_v1_8:ResourceManagerEnv',
    kwargs={
        'grid_size': 20,            #set to desired grid size
        'window_size': 300,         #set to desired window size
        'num_water_resources': 10,  #set to desired number of water resources
        'num_food_resources': 10,   #set to desired number of food resources
        'render_mode': 'human',      #set to desired render mode ('human' or 'rgb_array')
        'danger_mode': True,        #set to True to enable danger mode - spawn them lions!
        'max_episode_steps': 1000  #set to desired max episode steps
    },
)


Load the wanted Q-Table and Hyperparameters first

Overwrite Hyperparameters for exploiting most of the time

In [3]:

env = gym.make('Resource-Manager-v1-8')

#Hyperparameters:
#load the hyperparameters from the saved results
loaded_hyperparameters = np.genfromtxt('Results_11-12-2023_21-15-50_MoreReplayBuffer/Hyperparameters/hyperparameters.txt', dtype=None, delimiter='=', encoding=None)
hyperparameters = {}

#fill the hyperparameters dictionary
for key, value in loaded_hyperparameters:
    hyperparameters[key.strip()] = float(value)

learning_rate = hyperparameters['learning_rate']
discount_factor = hyperparameters['discount_factor']
#exploration_prob = hyperparameters['exploration_prob']
#epsilon_decay = hyperparameters['epsilon_decay']
#Do not load, keep exporation at minimum (0.1)
exploration_prob = 0.1
epsilon_decay = 1
replay_buffer_size = hyperparameters['replay_buffer_size']
batch_size = hyperparameters['batch_size']

#Load Q Table and Observation
loaded_q_table = np.loadtxt(f'Results_11-12-2023_21-15-50_MoreReplayBuffer/q_table.txt')
with open('Results_11-12-2023_21-15-50_MoreReplayBuffer/observation.txt', 'r') as file:
    loaded_observation_str = file.read()
loaded_observation_list = eval(loaded_observation_str)
loaded_observation = np.array(loaded_observation_list)


agent = QLearningAgent(env, learning_rate, discount_factor, exploration_prob, epsilon_decay, q_table=loaded_q_table)
num_episodes = 1

episode_rewards = []
episode_water_resources = []
episode_food_resources = []
episode_max_steps = []

for episode in range(num_episodes):
    observation = env.reset(custom_observation=loaded_observation)
    total_reward = 0

    while True:
            #select action using the Q-learning agent
            action = agent.select_action(observation)
            # Take a step in the environment
            next_observation, reward, done, _, _ = env.step(action)
            total_reward += reward
            
            observation = next_observation


            if done:
                break

    episode_rewards.append(total_reward)
    episode_water_resources.append(env.water_resource)
    episode_food_resources.append(env.food_resource)
    episode_max_steps.append(env.num_step)

    print(f"Episode {episode + 1}, Total Steps: {env.num_step}, Total Reward: {total_reward}, Total Water Resource: {env.water_resource}, Total Food Resource: {env.food_resource}")

c:\Users\denni\.conda\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (20, 20)
  logger.warn(
c:\Users\denni\.conda\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\denni\.conda\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
c:\Users\denni\.conda\envs\tf\lib\site-packages\gym\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` 

Episode 1, Total Steps: 223, Total Reward: -5557, Total Water Resource: -0.5, Total Food Resource: 109.5


: 